In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Keras란?

딥러닝 모델을 빌드하고 학습시키기 위한 tensorflow의 상위 수준 API이다. tensorflow 공식문서에서 keras의 장점을 세가지로 설명하였다.


- 사용자 친화적
    > 일반적인 사용 사례에 맞춰 최적화된 Keras의 인터페이스는 간단하고 일관성이 있습니다. Keras는 사용자 오류에 대해 명확하고 실행 가능한 피드백을 제공합니다.
    

- 모듈식 및 구성 가능
    > Keras 모델은 거의 제한 없이 구성 가능한 빌딩 블록을 함께 연결하는 방식으로 만들어집니다.
    

- 쉽게 확장 가능
    > 연구를 위한 새로운 아이디어를 표현하는 맞춤 빌딩 블록을 작성할 수 있습니다. 새로운 레이어, 측정항목, 손실 함수를 만들고 최첨단 모델을 개발합니다.

<img src="./education_images/4-1.png" alt="Drawing" style="width: 800px;"/>

tensorflow에서 제공하는 keras의 클래스 계층을 그림으로 나타내었다.

# Layer by Keras

이 전에 tensorflow로 구현하였던 fully connected layer를 가져왔다. 

keras를 사용하여 동일한 기능의 fully connected layer를 구현해보았다.

- 동일한 layer인지를 파악하기 위해 가중치의 초기값들을 1, 0으로 설정한다.  


- `unit` 은 output의 shape를 의미하며, kernel, bias는 각각 w, b에 해당한다.


- 초기값을 위해서 `keras.initializers`를 사용하였다.
  
  
- keras의 layer 클래스는 기본적으로 입력되는 데이터의 shape에 따라 가중치의 shape가 변하게 된다.  
   → 초기 선언시에는 trainable_variables이 존재하지 않는 것을 확인할 수 있다.
   
하지만 내장된 함수 이외의 연산이 필요하여 직접 만들어야하는 경우가 존재한다.     
이 때는 API를 그대로 사용하는 것이 아닌, 이 전의 클래스로 구현하는 방식을 그대로 사용한다. 아래의 예제를 보자.  

- 위의 fully connected layer를 keras의 class를 통해 구현하였다.


- `tf.Module` 클래스를 상속받는 대신 `tf.keras.layers.Layer`를 상속받아 사용하였다. 


- `Layer` 클래스는 layer 객체 호출시점에 수행되는 함수를  `__call__` 내에 구현하는 것이 아닌, `call`함수를 구현하여 사용한다. 


- `build` 함수는 해당 layer 객체 호출 시점에 `call` 함수를 수행하기 전 먼저 수행되며, 인자값으로 전달받는 `input_dims` 는 호출 시점에 전달받는 입력값의 shape를 의미한다. 해당 함수 내에서 내부 변수들을 초기화하면 입력의 shape에 대한 제약이 사라지게 된다.

# Model by Keras

- 이 전의 tensorflow로 구현하였던 모델은 `tf.Module` 을 상속받아 클래스로 구현하는 방식이었다. 


- keras에서는 Layer, Model 클래스를 구분지어두었다.


- 각 클래스에 따라 필요한 메소드를 미리 구현해두었다. Model 클래스에서만 모델의 구조를 볼 수 있는 `summary()`, 학습 루프를 진행하는 `fit()` 등의 함수를 사용할 수 있다. layer 클래스에 학습에 대한 함수는 필요하지 않기 때문이다.

# symbolic style

추상적인 방식으로의 구현으로, 내부 구체적인 구현을 직접하는 것이 아닌 이미 구현되어있는 API를 단순히 선형적으로 연결하여 모델을 구현하는 방식이다.   
이러한 방식에는 두가지 방법이 존재한다.

1. Functional API

2. Sequential API

## 1. Functional API

- 각 layer를 함수처럼 사용하여 입력에서 부터 순서대로 수행하도록 하는 방식으로 모델을 구현한다. 


- `tf.keras.Input` 클래스의 객체를 구현하여 입력값의 shape를 명시한다. 

## 2. Sequential API

- 가장 최하위 클래스인 Sequential 클래스를 사용하여 모델을 구현한다.   


- 단순히 layer들의 배열을 전달함을 통해 모델을 구현할 수 있으며, 모델의 입력또한 정의하지 않아도 된다.  
    → 위의 functional API의 경우, 입력을 명시해줘야 했다.

# imperative style (model subclassing) 


엄격한 방식으로의 구현으로, 이 전에 tensorflow에서 `tf.Module`의 하위 클래스를 선언하는 방식, keras에서 `tf.keras.layers.Layer`의 하위 클래스를 선언하여 layer를 구현하였던 방식과 동일하게 `tf.keras.Model`의 하위클래스를 선언하여 내부 구조까지 구현하는 방식이다. 

- 이전의 tensorflow에서 모델 구현방식과 동일하다. 


- 모델의 내부적인 구조까지 구체적으로 구현할 수 있다는 장점이 존재하지만, keras의 장점인 일관성을 가지는 인터페이스을 사용하지 못하는 단점이 존재한다. 
    → 다른 API와의 호환이 어려울 수 있다.

# Training by Keras

`tf.keras.Model` 클래스에서 제공하는 학습관련 함수들을 통해 학습을 쉽게 구현할 수 있다. 기존의 선형회귀 모델을 keras로 구현하여 학습과정을 keras로 구현해본다. (이 후의 여러가지 실습예제를 통해 경험할 것이기 때문에 간단히 하나의 예제만을 본다.)

1. `compile` 을 통해 모델에 대한 학습 루프를 정의한다. 사용되는 인자는 각각 optimizer, loss 를 정의한다.   

2. `fit` 을 통해 모델 학습을 진행한다. 사용되는 인자는 각각 epoch, batch를 정의한다.  

3. `evaluate`을 통해 모델을 평가한다.  

```python
    Model.compile(
        optimizer="rmsprop",
        loss=None,
        metrics=None,
        loss_weights=None,
        weighted_metrics=None,
        **kwargs
        )
```

### optimizer 

`tf.keras.optimizers`의 optimizer 객체를 정의한다.
옵티마이저는 loss를 최소화하는 결과를 빠르게 찾아낼 수 있도록 하는 객체이다. 여러 개의 객체가 존재하며 각각의 장단점이 존재한다.


### loss

`tf.keras.losses.Loss`의 Loss 객체를 정의한다.
이외에도 자신이 정의한 loss function을 사용할 수 있다. loss의 경우, 사용하는 데이터의 타입과 그 특성에 따라 정의해야한다.


### metrics

학습을 진행하는 시점에 확인할 지표들을 정의한다. 

```python
 metrics=['accuracy']
```

# 모델 저장하기

학습이 진행되고 난 이후 학습된 모델을 저장하기 위한 함수가 `tf.keras.Model` 클래스에 구현되어있다.